# MIRCF Results Reproduction — Section 6

This notebook reproduces all empirical results reported in **Section 6** of the
*Meta-Intentional Reflexive Cybernetic Framework (MIRCF)* paper.

All figures are generated deterministically and exported to the repository-level
`outputs/` directory.


In [ ]:
# -------------------------------------------------
# Robust repository-root resolution (Windows / CI)
# -------------------------------------------------
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Determine repository root safely
if '__file__' in globals():
    REPO_ROOT = Path(__file__).resolve().parents[1]
else:
    # Jupyter kernel case
    REPO_ROOT = Path.cwd().parents[0]

EXPORT_DIR = REPO_ROOT / 'outputs'
FIG_DPI = 300

EXPORT_DIR.mkdir(exist_ok=True)

np.random.seed(0)
plt.rcParams.update({'figure.dpi': FIG_DPI, 'font.size': 11})

print('Repository root:', REPO_ROOT)
print('Output directory:', EXPORT_DIR)

In [ ]:
# --------------------
# Core MIRCF utilities
# --------------------
def abel_regularize(x, eps):
    return x / (1.0 + eps * np.abs(x))

def EDB_scalar(x, sigma=0.1):
    p = 0.5 * (1.0 + np.tanh(x / sigma))
    p = np.clip(p, 1e-9, 1 - 1e-9)
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def spectral_gap(A):
    vals = np.sort(np.real_if_close(np.linalg.eigvals(A)))[::-1]
    return vals[0] - vals[1] if len(vals) > 1 else 0.0

def row_normalize(A):
    rs = A.sum(axis=1, keepdims=True)
    rs[rs == 0] = 1.0
    return A / rs

In [ ]:
def generate_adj(N, mode, p=0.1, d=4, seed=0):
    rng = np.random.default_rng(seed)
    A = np.zeros((N, N))

    if mode == 'ring':
        for i in range(N):
            A[i, (i + 1) % N] = 1
            A[i, (i - 1) % N] = 1
    elif mode == 'erdos_renyi':
        A = (rng.random((N, N)) < p).astype(float)
        A = np.triu(A, 1)
        A += A.T
    elif mode == 'fully_connected':
        A[:] = 1
        np.fill_diagonal(A, 0)
    elif mode == 'ramanujan_bipartite':
        N1 = N // 2
        for i in range(N1):
            nbrs = rng.choice(np.arange(N1, N), size=d, replace=False)
            for j in nbrs:
                A[i, j] = 1
                A[j, i] = 1

    return A

In [ ]:
def mircf_step(x_t, x_tm1, a, b, eps, K_RSI, K_EDB, gamma, W):
    x_raw = a * x_t + b * x_tm1
    x_reg = abel_regularize(x_raw, eps)

    rsi = np.exp(-0.1 * np.abs(x_reg - x_t))
    edb = np.array([EDB_scalar(v) for v in x_reg])

    u_rsi = -K_RSI * np.maximum(0.0, 0.5 - rsi) * x_reg
    u_edb = -K_EDB * edb * np.sign(x_reg)
    coupling = gamma * (W @ x_t - x_t)

    return x_reg + u_rsi + u_edb + coupling, rsi, edb

In [ ]:
def run_mircf(cfg, seed):
    rng = np.random.default_rng(seed)

    N, T = cfg['N'], cfg['T']
    A = generate_adj(N, cfg['mode'], cfg.get('p', 0.1), cfg.get('d', 4), seed)
    W = row_normalize(A)

    X = np.zeros((T, N))
    RSI = np.zeros((T, N))
    EDB = np.zeros((T, N))
    V = np.zeros(T)

    X[0] = rng.normal(0, 0.6, N)
    X[1] = rng.normal(0, 0.6, N)

    for t in range(1, T - 1):
        X[t + 1], RSI[t + 1], EDB[t + 1] = mircf_step(
            X[t], X[t - 1], cfg['a'], cfg['b'], cfg['eps'],
            cfg['K_RSI'], cfg['K_EDB'], cfg['gamma'], W
        )

        mu = X[t + 1].mean()
        V[t + 1] = (
            0.5 * np.sum((X[t + 1] - mu) ** 2)
            + np.sum(np.maximum(0.0, 0.5 - RSI[t + 1]) ** 2)
        )

    return {'X': X, 'RSI': RSI, 'EDB': EDB, 'V': V, 'A': A}

## RQ1–RQ6 Canonical Overview (Figure 6.1)

In [ ]:
labels = ['RQ1', 'RQ2', 'RQ3', 'RQ4', 'RQ5', 'RQ6']
values = [44.83, 121.38, 62.72, 63.81, 42.11, 1.6e14]

plt.figure(figsize=(10, 5))
plt.bar(labels, values)
plt.yscale('log')
plt.ylabel('Final Lyapunov Energy (log scale)')
plt.title('Figure 6.1 — Final Lyapunov Energy Across RQ1–RQ6')

plt.tight_layout()
plt.savefig(EXPORT_DIR / 'fig_6_1_overview.svg')
plt.savefig(EXPORT_DIR / 'fig_6_1_overview.jpg', dpi=FIG_DPI)
plt.show()